In [1]:
import numpy as np

import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app

In [2]:
for ii in range(mm.Platform.getNumPlatforms()):
    print(mm.Platform.getPlatform(ii).getName())

Reference
CPU
OpenCL


In [3]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

K = 100.0 * unit.kilocalories_per_mole/unit.angstrom**2
mass = (12.010*15.999)/(12.010+15.999) * unit.amu # C-O atoms
num_degrees_of_freedom = 1

In [4]:
step_size       = 0.001*unit.picoseconds
num_steps       = 30000
saving_period   = 100
temperature = 300*unit.kelvin
friction    = 1/unit.picosecond
platform_name='CPU'  #platform:     A platform (CPU, OpenCL, CUDA, or reference); default is platform=OpenCL"

initial_positions = unit.Quantity(np.zeros([1,3], np.float32), unit.angstroms)

"""
T = sqrt(m/K)
timestep smaller than ~ T/10
standard deviation in each dimension sigma= (kT / K)^(1/2)
Expectation and standard deviation of the potential energy of a 3D harmonic oscillator is (3/2)kT
"""

In [5]:
num_checks     = int(num_steps/saving_period)
times      = unit.Quantity(np.zeros([num_checks], np.float32), unit.picoseconds)
positions  = unit.Quantity(np.zeros([num_checks,3], np.float32), unit.angstroms)
velocities = unit.Quantity(np.zeros([num_checks,3], np.float32), unit.angstroms/unit.picosecond)
potential_energies   = unit.Quantity(np.zeros([num_checks], np.float32), unit.kilocalories_per_mole)
kinetic_energies     = unit.Quantity(np.zeros([num_checks], np.float32), unit.kilocalories_per_mole)

In [6]:
system = mm.System()
system.addParticle(mass)

if num_degrees_of_freedom==1:
    force = mm.CustomExternalForce('(K/2.0) * (x^2)')
elif num_degrees_of_freedom==2:
    force = mm.CustomExternalForce('(K/2.0) * (x^2 + y^2)')
elif num_degrees_of_freedom==3:
    force = mm.CustomExternalForce('(K/2.0) * (x^2 + y^2 + z^2)')

In [7]:
force.addGlobalParameter('K', K)
force.addParticle(0, [])
system.addForce(force)

0

In [8]:
integrator = mm.LangevinIntegrator(temperature, friction, step_size)
platform = mm.Platform.getPlatformByName(platform_name)


In [9]:
context = mm.Context(system, integrator, platform)
context.setPositions(initial_positions)
context.setVelocitiesToTemperature(temperature)

In [10]:
state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
times[0] = state.getTime()
positions[0] = state.getPositions()[0]
velocities[0] = state.getVelocities()[0]
kinetic_energies[0]=state.getKineticEnergy()
potential_energies[0]=state.getPotentialEnergy()

In [11]:
for ii in range(num_checks):

    context.getIntegrator().step(saving_period)
    state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
    times = state.getTime()
    positions[ii] = state.getPositions()[0]
    velocities[ii] = state.getVelocities()[0]
    kinetic_energies[ii]=state.getKineticEnergy()
    potential_energies[ii]=state.getPotentialEnergy()



In [12]:
times

Quantity(value=30.00000000001368, unit=picosecond)

In [14]:
plt.plot(positions)

NameError: name 'plt' is not defined

In [ ]:
velocities

In [ ]:
kinetic_energies

In [ ]:
potential_energies